# Authentication

# Authentication options

To authenticate your account with the godel python SDK you have two options: you may retrieve your token from your profile page in the dApp or you may programmatically generate a key using your wallet address and secret keys.

# Retrieve JWT Token from dApp UI

Easiest option: Follow the directions at [Authentication](https://docs.golden.xyz/api/authentication) to skip the programmatic authentication and plainly retrieve your JWT token from https://dapp.golden.xyz. 

## Prerequisite

[QuickStart](https://docs.golden.xyz/godel-python-sdk/quickstart)

This guide requires the installation of [Web3.py](https://github.com/ethereum/web3.py).

You can do this with `pip install godel[web3]` and comes pre-installed if using the godel docker image.

### 1. Connect to Golden Web3 API
Let's connect the python wrapper to the Golden GraphQL API.

You can run your basic queries against the API without authenticating, but will be restricted from mutations and certain queries until you connect using your USER_ID (your wallet address) and PRIVATE_KEY (wallet secret).

Note: If you encounter errors with connection, ensure that the wallet being used has been connected to your profile at

In [ ]:
USER_ID = #"0x01234...YOUR_WALLET_ID_HERE"
PRIVATE_KEY = #"YOUR_WALLET_SECRET_HERE"

In [ ]:
from godel import GoldenAPI

DAPP_URL = "https://dapp.golden.xyz/graphql"
goldapi = GoldenAPI(url=DAPP_URL)

### 2. Authenticate and set JWT

This will set the JWT token key in your GoldenAPI object so you should have permission to hit all endpoints after running the below.

In [ ]:
# Retrieve one-off nonce from GraphQL API
message_response = goldapi.get_authentication_message(user_id=USER_ID)
message_response

# Sign and verify nonce with your wallet's private key (KEEP THIS SECURE)
from web3.auto import w3
from eth_account.messages import encode_defunct

message_string = message_response["data"]["getAuthenticationMessage"]["string"]
message = encode_defunct(text=message_string)
signed_message = w3.eth.account.sign_message(message, private_key=PRIVATE_KEY)
signature = signed_message.signature.hex()

# Authenticate with Golden's API and you'll recieve a jwt bearer token
auth_response = goldapi.authenticate(
    user_id=USER_ID,
    signature=signature
)

jwt_token = auth_response["data"]["authenticate"]["jwtToken"]

# Set JWT token to verify your wallet/role and unlock permissions to the rest of the API
goldapi.set_jwt_token(jwt_token=jwt_token)

print("Your JWT Token:\n", jwt_token)

#### Convenience Method for Authentication

Runs the above authentication process in a secure and convenient method.

In [ ]:
goldapi.get_authentication_token(
    user_id=USER_ID,
    wallet_private_key=PRIVATE_KEY
)

#### Retrieve JWT Token from GUI

Follow the directions at https://docs.golden.xyz/guides/authentication to skip the programmatic authentication and plainly retrieve your JWT token from https://dapp.golden.xyz.

### 3. Test with Graphql Calls

Once authenticated, you'll now have permission to search, disambiguate, validate, and submit data to the Golden Protocol.

In [ ]:
goldapi.entity_search("golden")